In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pyactlike

# Example for Cobaya (version < 2.1)

This example demonstrates how to use the ACT likelihood on versions of Cobaya less than 2.1, such as the current stable release.

In [3]:
# Test the Cobaya interface to the ACT likelihood.
from cobaya.yaml import yaml_load
from cobaya.model import get_model

# need to define a wrapper for this external likelihood and ask for C_ell
like = pyactlike.ACTPowerSpectrumData()
def external_act_like(yp2=1.0, 
        _theory={'Cl': {
            'tt': like.tt_lmax, 
            'te': like.tt_lmax, 
            'ee': like.tt_lmax
        }}, 
        _derived={}):
    Cls = _theory.get_Cl(ell_factor=True)
    return like.loglike(
        Cls['tt'][2: like.tt_lmax+1], 
        Cls['te'][2: like.tt_lmax+1], 
        Cls['ee'][2: like.tt_lmax+1], yp2)

# set up our model
info = {
    'likelihood': {'act': external_act_like},
    'theory': {'camb': {'stop_at_error': True}},
    'params': {
        'ns': {
            'prior': {
                'min': 0.8,
                'max': 1.2
            },
            'ref': {
                'dist': 'norm',
                'loc': 0.965,
                'scale': 0.004
            },
            'proposal': 0.002,
            'latex': r'n_\mathrm{s}'
        },
        'yp2':{
            'prior': {
                'min': 0.9,
                'max': 1.1
            }
        },
        
        'As': 2.1e-09,
        'H0': 68,
        'mnu': 0.06,
        'nnu': 3.046,
        'num_massive_neutrinos': 1,
        'ombh2': 0.022,
        'omch2': 0.12,
        'tau': 0.07,
        
        
    },
    
    'sampler': {
        'mcmc': {
            'proposal_scale': 2.4
        }
    }
}
model = get_model(info)

[model] *WARNING* Ignored blocks/options: ['sampler']
[act] Initialised external likelihood.
[camb] Importing *global* CAMB.
[likelihood] The theory code will compute the following products, requested by the likelihoods: ['Cl']


In [4]:
fiducial_params = {
    'ns': 0.97,
    'yp2':1.0
}

In [5]:
model.loglike(fiducial_params)[0]

-188.21785329683718

In [ ]:
from cobaya.run import run
updated_info, products = run(info)